# Create outer punches from subtracting from outer contours

In [1]:
# I will need a function to calculate the length and width 

In [8]:
# general functions
def get_length_width(outer_contour):
    print(f"len entities:{len(outer_contour[0])}")

    if len(outer_contour[0]) <10:
        for entity in outer_contour:
            print(entity)
            p1 = (entity[1],entity[2],entity[3])
            p2 = (entity[5],entity[6],entity[7])
            dist = get_dist(p1,p2)
            print(dist)
            entity.append("dist")
            entity.append(dist)
            

    width = 0
    width_entity = None
    length = 0
    length_entity = None
    print(f"outer contourrrrrrrrrrrrr:{outer_contour}")
    for entity in outer_contour:
        if entity[8]=='LINE':
            change_in_x = entity[5] -entity[1]
            if length < change_in_x:
                print(f"length found:{entity}")
                length = entity[10]
                length_entity = entity
    
    for entity in outer_contour:
        if entity[8]=='LINE':
            change_in_y = entity[7] - entity[3]
            print(f"change in y:{change_in_y}")
            if width < change_in_y: #checkingg if entity is equal to length entity not the length itself in mm
                print(f"width found:{entity}")
                width = entity[10]
                width_entity = entity


    if width_entity == None:
        width_entity = length_entity
        width = length
    print(f"length:{length}, length entity:{length_entity}\n width:{width}, width_entity:{width_entity}")
    return length,length_entity,width,width_entity


import math
def get_dist(p1,p2):
    l = math.dist(p1,p2)
    return l 


In [9]:
outer_contours = [
    ['VERTEX_POINT', -142, 6, -88, 'VERTEX_POINT', -142, 6, 62, 'LINE'], # width
    ['VERTEX_POINT', -142, 6, 62, 'VERTEX_POINT', 58, 6, 62, 'LINE'], # length
    ['VERTEX_POINT', 58, 6, 62, 'VERTEX_POINT', 58, 6, -88, 'LINE'], # width
    ['VERTEX_POINT', 58, 6, -88, 'VERTEX_POINT', -142, 6, -88, 'LINE'], # length 
]


In [ ]:
length,length_entity, width,width_entity = get_length_width(outer_contours)

In [11]:
length

200.0

In [12]:
length_entity

['VERTEX_POINT', -142, 6, 62, 'VERTEX_POINT', 58, 6, 62, 'LINE', 'dist', 200.0]

In [13]:
width

150.0

In [14]:
width_entity

['VERTEX_POINT',
 -142,
 6,
 -88,
 'VERTEX_POINT',
 -142,
 6,
 62,
 'LINE',
 'dist',
 150.0]

In [15]:
# create a function to get list of two entities have similar points 

In [20]:
def get_points(entity):
    p1 = (entity[1],entity[2],entity[3])
    p2 = (entity[5],entity[6],entity[7])
    return [p1,p2]

def check_similar_entities(e1,e2):
    e1_points = get_points(e1)
    e2_points = get_points(e2)
    
    for p in e1_points:
        if p in e2_points:
            return True
    return False

In [39]:
def get_corners(outer_contours):
    corners = [] 
    for e1 in outer_contours:
        for e2 in outer_contours:
            if e1 == e2:
                continue
            if check_similar_entities(e1,e2) and ([e1,e2] not in corners and [e2,e1] not in corners):
                corners.append([e1,e2])
    return corners

In [41]:
corners= get_corners(outer_contours)

In [42]:
len(corners)

4

In [47]:
for corner in corners:
    e1= corner[0]
    e2= corner[1]
    
    e1_points = get_points(e1)
    e2_points = get_points(e2)
    
    print(e1_points,e2_points)
    print(f"slope e1:{identify_line(e1_points)}")
    print(f"slope e2:{identify_line(e2_points)}")

[(-142, 6, -88), (-142, 6, 62)] [(-142, 6, 62), (58, 6, 62)]
slope e1:Division by zero
slope e2:0.0
[(-142, 6, -88), (-142, 6, 62)] [(58, 6, -88), (-142, 6, -88)]
slope e1:Division by zero
slope e2:-0.0
[(-142, 6, 62), (58, 6, 62)] [(58, 6, 62), (58, 6, -88)]
slope e1:0.0
slope e2:Division by zero
[(58, 6, 62), (58, 6, -88)] [(58, 6, -88), (-142, 6, -88)]
slope e1:Division by zero
slope e2:-0.0


In [46]:
def identify_line(points):
    # cases 
    # 1. if slope is Division by zero then the line is vertical 
    # 2. if slope is zero then the line is horizontal
    # 3. if slope is +ve then the line is increasing forward
    # 4. if slope is -ve then the line is decreasing 
    p1 = points[0]
    p2 = points[1]
    try:
        slope = (p2[1]-p1[1])/(p2[0]-p1[0])
    except:
        return "Division by zero"
    return slope